In [ ]:
#|default_exp deck

In [ ]:
#|hide
from nbprocess.showdoc import *

# Deck
> Playing Cards

In [ ]:
#|export
from deck_of_cards.card import Card
import random

In [ ]:
#|export
class Deck:
    """Represents a deck of cards.
    Attributes:
      cards: list of Card objects.
    """
    
    def __init__(self):
        "Initializes the Deck with 52 cards."
        self.cards = [Card(suit, rank) for suit in range(4) for rank in range(1, 14)]

    def __str__(self):
        "Returns a string representation of the deck."
        return '\n'.join([str(card) in self.cards])
    
    def add_card(self, card:Card):
        "Adds a card to the deck."
        self.cards.append(card)

    def remove_card(self, card):
        "Removes a card from the deck or raises exception if it is not there."
        self.cards.remove(card)

    def pop_card(self, i=-1):
        "Removes and returns a card from the deck."
        return self.cards.pop(i)

    def shuffle(self):
        """Shuffles the cards in this deck."""
        random.shuffle(self.cards)

    def sort(self):
        """Sorts the cards in ascending order."""
        self.cards.sort()

    def move_cards(self, 
                   hand, # destination Hand object 
                   num:int # integer number of cards to move
                  ):
        "Moves the given number of cards from the deck into the Hand."
        for i in range(num): hand.add_card(self.pop_card())

A Deck of cards is a collection of `Card` objects:

In [ ]:
deck = Deck()
assert isinstance(deck.pop_card(), Card)

You can show the docs for methods by calling `show_doc`. For example, the code `show_doc(Deck.remove_card)` produces the following documentation:

In [ ]:
show_doc(Deck.remove_card)

---

#### Deck.remove_card

>      Deck.remove_card (card)

Removes a card from the deck or raises exception if it is not there.

If we remove a card from the Deck we can verify that it no longer exists:

In [ ]:
card23 = Card(2, 3)
deck.remove_card(card23)

assert card23 not in deck.cards



However, another card that we haven't removed, such as the `10 of hearts` will still be in the Deck of cards because we haven't removed it:


In [ ]:
c = Card(2,10)
assert c in deck.cards
c

10 of Hearts

## Drawing Cards With Replacement

Let's try something fun with our deck of cards, drawing a card with replacement:

In [ ]:
#|export
#|export
def draw_n(n, replace=True):
    "draw n cards"
    d = Deck()
    d.shuffle()
    if replace: return [d.cards[random.choice(range(len(d.cards)))] for _ in range(n)]
    else: return d.cards[:n]

In [ ]:
sample = draw_n(10)
sample

[7 of Diamonds,
 Jack of Clubs,
 Queen of Hearts,
 9 of Diamonds,
 2 of Clubs,
 2 of Spades,
 Queen of Hearts,
 10 of Clubs,
 Queen of Spades,
 4 of Spades]

In [ ]:
#|hide
assert len(sample) == 10

## Visualizing the results

This isn't terribly interesting from a statistical perspective.  However, its an example of how you can include visualizations in your nbdev projects!  Notice how we are hiding just the input with `#|hide_input`:

In [ ]:
#|hide_input
import altair as alt
import pandas as pd

alt.data_transformers.disable_max_rows()
sampledf = pd.DataFrame([{'suit': c.suit_nm, 'rank': c.rank_nm } for c in draw_n(5000)])

rect = alt.Chart(sampledf).mark_rect().encode(
    alt.X('suit:N'),
    alt.Y('rank:N'),
    alt.Color('count()',
        scale=alt.Scale(scheme='blueorange'),
        legend=alt.Legend(title='Total Records')
    )
).properties(
    width=550,
    height=550
)
rect

alt.Chart(...)

## Create a CLI

We can create a CLI with `@call parse`

In [ ]:
#|export
from sys import stdout
from pathlib import Path
from fastcore.foundation import L
from fastcore.script import call_parse

In [ ]:
#|export
@call_parse
def draw_cards(n:int, #number of cards to draw
               replace:bool=True, # whether or not draw with replacement
               outfile:str=None #output file, defaults to stdout
               ):
    "Draw `n` cards optionally with replacement."
    cards = draw_n(n, replace=replace)
    strcards = '\n'.join(L(cards).map(str))
    stdout.write(strcards) if outfile is None else Path(outfile).write_text(strcards)

In [ ]:
draw_cards(10, outfile='sample.txt')
!cat sample.txt

6 of Diamonds
Jack of Clubs
Jack of Clubs
10 of Diamonds
Jack of Clubs
Ace of Spades
3 of Spades
Ace of Diamonds
Jack of Spades
Queen of Clubs